<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_HW5/ECGR_4105_HW5_Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 2

In [627]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [628]:
#Calling the dataset
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/4105 Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [629]:
#Applying a binary map
#mapping the variables to a binary output
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

housing.head()
#housing.dtypes

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [630]:
#Splitting the Dataset
x2_train, x2_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)
x2_train.head()

#Splitting Variables
y2_train = x2_train.pop("price")
y2_val = x2_val.pop("price")

#print(x2_train)
#print(y2_train)
#x2_train.head()
#print(y2_val)

In [631]:
#Standardizing the Dataset

standard = StandardScaler()

vars2 = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

x2_train_stan = x2_train
x2_train_stan[vars2] = standard.fit_transform(x2_train_stan[vars2])

x2_val_stan = x2_val
x2_val_stan[vars2] = standard.fit_transform(x2_val_stan[vars2])
#x2_val_stan.head()

In [632]:
#Stacking them there inputs
x2_train_stan_numpy = np.c_[np.ones((len(y2_train), 1)), x2_train_stan[vars2]]
x2_val_stan_numpy = np.c_[np.ones((len(y2_val), 1)), x2_val_stan[vars2]]

#I am become tensor destroyer of deep learning
x2_train_ten = torch.tensor(x2_train_stan_numpy)
x2_val_ten = torch.tensor(x2_val_stan_numpy)
y2_train_ten = torch.tensor(y2_train.values)
y2_val_ten = torch.tensor(y2_val.values)


y2_train_mean = y2_train_ten.float().mean()
y2_train_std = y2_train.std()
y2_train_stan_ten = (y2_train_ten-y2_train_mean) / y2_train_std

y2_val_mean = y2_val_ten.float().mean()
y2_val_std = y2_val.std()
y2_val_stan_ten = (y2_val_ten-y2_val_mean) / y2_val_std

In [633]:
#Getting lost and starting my modeling career

#defining the model
def model(t_u, w5, w4, w3, w2, w1, b):
  return w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#Setting up the initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [634]:
#Time for an anime training arc

#defining a training loop
def training_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:  # <1>
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params


#SGD Number 1 learning rate of 0.00001
learning_rate_SGD1 = 0.00001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD1 = optim.SGD([params], lr=learning_rate_SGD1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD1,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26248572928235.8672, Validation loss: 24880934228406.0352

Epoch 3, Training loss: 26135719257251.2617, Validation loss: 24712263458587.7617

Epoch 500, Training loss: 24101121059367.8984, Validation loss: 22579418248648.8203

Epoch 1000, Training loss: 22844945594771.5781, Validation loss: 21388146851785.9492

Epoch 1500, Training loss: 21968032839487.1484, Validation loss: 20577481812830.6641

Epoch 2000, Training loss: 21296522199705.6250, Validation loss: 19959627431284.1914

Epoch 2500, Training loss: 20740636009235.9102, Validation loss: 19442648150641.4492

Epoch 3000, Training loss: 20253199008904.3203, Validation loss: 18981388764453.2031

Epoch 3500, Training loss: 19808882743484.0625, Validation loss: 18553536828546.1445

Epoch 4000, Training loss: 19393729983828.9922, Validation loss: 18147942569271.1445

Epoch 4500, Training loss: 18999810903000.3750, Validation loss:

tensor([-38222.3125, 161046.7188, 133949.1406, 279947.9688,  63465.4453,
        402210.6562], requires_grad=True)

In [635]:
#SGD Number 2 learning rate of 0.000001
learning_rate_SGD2 = 0.000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr=learning_rate_SGD2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26444054138494.7539, Validation loss: 25154006707764.3203

Epoch 3, Training loss: 26419768960376.4844, Validation loss: 25120837976976.1016

Epoch 500, Training loss: 25828047200433.5000, Validation loss: 24283400526493.9141

Epoch 1000, Training loss: 25595290905301.1602, Validation loss: 24049686945979.8438

Epoch 1500, Training loss: 25374286395875.2852, Validation loss: 23828861554546.7812

Epoch 2000, Training loss: 25164276964070.6406, Validation loss: 23620042532493.1523

Epoch 2500, Training loss: 24964557125485.6484, Validation loss: 23422409962052.2773

Epoch 3000, Training loss: 24774466931196.2656, Validation loss: 23235199221830.1914

Epoch 3500, Training loss: 24593391340893.7070, Validation loss: 23057699743155.4062

Epoch 4000, Training loss: 24420755350067.2617, Validation loss: 22889248893728.3047

Epoch 4500, Training loss: 24256023947274.4688, Validation loss:

tensor([-9837.5850, 68577.5859, 32069.6055, 41701.6172, 10591.3525, 45407.4492],
       requires_grad=True)

In [636]:
#SGD Number 3 learning rate of 0.0000001
learning_rate_SGD3 = 0.0000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD3 = optim.SGD([params], lr=learning_rate_SGD3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD3,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26467168723914.5000, Validation loss: 25185440100266.0430

Epoch 3, Training loss: 26464575165315.8281, Validation loss: 25181919666243.5781

Epoch 500, Training loss: 26042200899149.0820, Validation loss: 24551615906515.4336

Epoch 1000, Training loss: 26013485609373.6680, Validation loss: 24483103000140.8711

Epoch 1500, Training loss: 25994932781339.6211, Validation loss: 24454954089681.8867

Epoch 2000, Training loss: 25972395014167.3438, Validation loss: 24429776853721.8555

Epoch 2500, Training loss: 25948419733555.7070, Validation loss: 24404942526320.1133

Epoch 3000, Training loss: 25924124775815.7812, Validation loss: 24380265449717.9805

Epoch 3500, Training loss: 25899837440596.7383, Validation loss: 24355731599531.1602

Epoch 4000, Training loss: 25875645905472.2891, Validation loss: 24331339133382.9336

Epoch 4500, Training loss: 25851572350414.2539, Validation loss:

tensor([4901.0444, 9497.7920, 3939.9980, 4558.1758, 1171.4553, 4643.4336],
       requires_grad=True)

In [637]:
#SGD Number 4 learning rate of 0.00000001
learning_rate_SGD4 = 0.00000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD4 = optim.SGD([params], lr=learning_rate_SGD4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD4,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469515847374.9062, Validation loss: 25188624701339.5078

Epoch 3, Training loss: 26469254803564.3555, Validation loss: 25188270578891.6211

Epoch 500, Training loss: 26356251028019.1680, Validation loss: 25033294149738.5195

Epoch 1000, Training loss: 26270713038640.6836, Validation loss: 24913013326849.0781

Epoch 1500, Training loss: 26206554909862.2578, Validation loss: 24820161096778.2305

Epoch 2000, Training loss: 26158543640655.4141, Validation loss: 24748348563351.2617

Epoch 2500, Training loss: 26122696143321.5938, Validation loss: 24692674312549.7031

Epoch 3000, Training loss: 26095985128263.6211, Validation loss: 24649379377689.5352

Epoch 3500, Training loss: 26076112382131.9219, Validation loss: 24615580052890.6172

Epoch 4000, Training loss: 26061336398461.4688, Validation loss: 24589064805497.1055

Epoch 4500, Training loss: 26050339625769.8633, Validation loss:

tensor([4851.5259, 1914.0765,  617.1115,  510.6638,  132.4113,  468.9208],
       requires_grad=True)

In [643]:
#Adam Number 1 learning rate of 0.1
learning_rate_Adam1 = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam1 = optim.Adam([params], lr=learning_rate_Adam1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam1,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469755792759.0898, Validation loss: 25188951589767.7617

Epoch 3, Training loss: 26469734546773.3594, Validation loss: 25188924171014.7188

Epoch 500, Training loss: 26459236805234.9883, Validation loss: 25175371147124.4766

Epoch 1000, Training loss: 26448797393914.0859, Validation loss: 25161882738131.2617

Epoch 1500, Training loss: 26438473434667.4961, Validation loss: 25148532517520.6133

Epoch 2000, Training loss: 26428259008272.6562, Validation loss: 25135312701162.4062

Epoch 2500, Training loss: 26418149099223.1367, Validation loss: 25122216684053.3672

Epoch 3000, Training loss: 26408139533130.0312, Validation loss: 25109238945511.4570

Epoch 3500, Training loss: 26398227178508.4883, Validation loss: 25096375351395.1758

Epoch 4000, Training loss: 26388409377433.4258, Validation loss: 25083622351190.1602

Epoch 4500, Training loss: 26378684444639.0234, Validation loss:

tensor([494.6991, 496.6452, 497.5805, 499.8134, 499.7426, 499.8969],
       requires_grad=True)

In [639]:
#Adam Number 2 learning rate of 00.01
learning_rate_Adam2 = 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam2 = optim.Adam([params], lr=learning_rate_Adam2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam2,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469774914470.9961, Validation loss: 25188976267024.0977

Epoch 3, Training loss: 26469772789824.6055, Validation loss: 25188973525093.5703

Epoch 500, Training loss: 26468717457159.1719, Validation loss: 25187611528182.4062

Epoch 1000, Training loss: 26467656979265.7500, Validation loss: 25186242783260.2930

Epoch 1500, Training loss: 26466597665943.7109, Validation loss: 25184875431694.4102

Epoch 2000, Training loss: 26465539458491.5703, Validation loss: 25183509395968.9414

Epoch 2500, Training loss: 26464482314047.3945, Validation loss: 25182144615085.9766

Epoch 3000, Training loss: 26463426215871.3164, Validation loss: 25180781069912.0508

Epoch 3500, Training loss: 26462371127841.8086, Validation loss: 25179418714643.0664

Epoch 4000, Training loss: 26461317028673.6562, Validation loss: 25178057513193.9375

Epoch 4500, Training loss: 26460263828800.1289, Validation loss:

tensor([50.9383, 50.9556, 50.9661, 50.9908, 50.9887, 49.9974],
       requires_grad=True)

In [640]:
#Adam Number 3 learning rate of 0.001
learning_rate_Adam3 = 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam3 = optim.Adam([params], lr=learning_rate_Adam3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam3,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469776826646.1758, Validation loss: 25188978734752.5273

Epoch 3, Training loss: 26469776614171.3750, Validation loss: 25188978460546.2930

Epoch 500, Training loss: 26469671025909.3594, Validation loss: 25188842194864.9219

Epoch 1000, Training loss: 26469564813776.5820, Validation loss: 25188705122781.7305

Epoch 1500, Training loss: 26469458610752.3320, Validation loss: 25188568061221.4570

Epoch 2000, Training loss: 26469352416747.5977, Validation loss: 25188431010081.3281

Epoch 2500, Training loss: 26469246232331.3516, Validation loss: 25188293969923.8125

Epoch 3000, Training loss: 26469140056936.3242, Validation loss: 25188156940188.1289

Epoch 3500, Training loss: 26469033890574.8086, Validation loss: 25188019920889.7070

Epoch 4000, Training loss: 26468927733219.9492, Validation loss: 25187882911995.4297

Epoch 4500, Training loss: 26468821584882.8945, Validation loss:

tensor([5.9997, 5.9997, 5.9997, 5.9997, 5.9997, 4.9998], requires_grad=True)

In [641]:
#Adam Number 4 learning rate of 0.0001
learning_rate_Adam4 = 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam4 = optim.Adam([params], lr=learning_rate_Adam4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam4,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469777017871.1055, Validation loss: 25188978981535.0273

Epoch 3, Training loss: 26469776996621.2070, Validation loss: 25188978954111.2422

Epoch 500, Training loss: 26469766435460.9844, Validation loss: 25188965324538.7695

Epoch 1000, Training loss: 26469755810635.7891, Validation loss: 25188951612793.7656

Epoch 1500, Training loss: 26469745185900.9180, Validation loss: 25188937901153.0820

Epoch 2000, Training loss: 26469734561256.2148, Validation loss: 25188924189616.5938

Epoch 2500, Training loss: 26469723936701.7539, Validation loss: 25188910478184.3555

Epoch 3000, Training loss: 26469713312308.7266, Validation loss: 25188896766926.7539

Epoch 3500, Training loss: 26469702688005.9492, Validation loss: 25188883055773.4102

Epoch 4000, Training loss: 26469692063793.4141, Validation loss: 25188869344724.3242

Epoch 4500, Training loss: 26469681439671.1211, Validation loss:

tensor([1.5001, 1.5001, 1.5001, 1.5001, 1.5001, 0.5000], requires_grad=True)